In [1]:
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms

In [2]:
transform=transforms.Compose([transforms.Resize(size=(224,224)),transforms.ToTensor()])
dataset=torchvision.datasets.ImageFolder(root='img_data_mel',transform=transform)
print(len(dataset))

1000


In [3]:
num_train=(int)(0.9*len(dataset))
num_test=len(dataset)-num_train
trainset,testset=torch.utils.data.random_split(dataset, lengths=(num_train,num_test))

In [4]:
trainloader=DataLoader(trainset,batch_size=4,shuffle=True,num_workers=2)
testloader=DataLoader(testset,batch_size=4,shuffle=True,num_workers=2)

In [5]:
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

In [6]:
model=models.vgg16_bn(pretrained=True)

In [7]:
num_classes=10
model.classifier[6]=nn.Linear(4096,num_classes)
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256, kernel_size=(3, 3)

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [10]:
model=model.to(device)

In [11]:
import torch.optim as optim 
criterion=nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(),lr=0.001,momentum=0.9)

In [12]:
num_epochs=10

In [13]:
for epoch in range(num_epochs):
    running_loss=0
    for i,data in enumerate(trainloader,0):
        inputs,labels=data #getting the data
        optimizer.zero_grad() #zeroing all gradients
        outputs=model(inputs)
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step() #step() performs the required update
        running_loss+=loss.item()
        if(i%2000==1999):
            print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0
print('Finished training.')

KeyboardInterrupt: 

In [ ]:
correct=0
total=0
with torch.no_grad():
    for data in testloader:
        images,labels=data
        outputs=model(images)
        _,predicted=torch.max(outputs.data,1)
        total+=labels.size(0)
        correct+=(predicted==labels).sum().item()
accuracy=100*correct/total
print("Accuracy: ",accuracy)
  